## Notebook Contains

A Rag Agent using `Cewai`

## Importing Libraries

In [1]:
# Importing Necessaet Libraries
import os
import sys
import io
from crewai import LLM, Agent, Task, Crew
# from langchain_huggingface import HuggingFaceEmbeddings
# from langchain_chroma import Chroma
# from langchain.text_splitter import RecursiveCharacterTextSplitter

from crewai.tools import BaseTool

HTTPSConnectionPool(host='telemetry.crewai.com', port=4319): Max retries exceeded with url: /v1/traces (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x00000186F5A0BF20>, 'Connection to telemetry.crewai.com timed out. (connect timeout=30.0)'))
HTTPSConnectionPool(host='telemetry.crewai.com', port=4319): Max retries exceeded with url: /v1/traces (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x00000186FA0EA060>, 'Connection to telemetry.crewai.com timed out. (connect timeout=30.0)'))


In [2]:
# --- Configuration --- #
MODEL_NAME = "ollama/openhermes:latest"
# DATA_FILE = "Data"
# DATA_FILE = os.path.join(DATA_FILE, "HR_Policy.txt")
# VECTORSTORE_DIR = "hr_policy_vectorstore"
# EMBEDDING_MODEL = "all-MiniLM-L6-v2"
# CHUNK_SIZE = 1000
# CHUNK_OVERLAP = 150


## Creating Vector Database

In [3]:
# # Loading and Chunking the Data
# def load_and_chunk_documents():
#     """"Loads the HR policy data from the TXT file and splits it into smaller chunks."""
#     print("\t --- Step 1: Loading and Chunking Data ---")

#     if not os.path.exists(DATA_FILE):
#         print(f"Data file {DATA_FILE} does not exist.")
#         return None

#     # Read the text file
#     with open(DATA_FILE, 'r', encoding='utf-8') as file:
#         full_text = file.read()
    
#     # Split the text into chunks
#     documents = [full_text]

#     text_splitter = RecursiveCharacterTextSplitter(
#         chunk_size= CHUNK_SIZE,
#         chunk_overlap= CHUNK_OVERLAP
#     )

#     chunks = text_splitter.create_documents(documents)
#     print(f"Data loaded and split into {len(chunks)} chunks.")
#     return chunks
    

In [4]:
# # Creating and embedding the Vector Store
# def create_and_store_embeddings(chunks):
#     """Creates a vector store from the chunks of data."""
#     print("\t --- Step 2: Creating Vector Store ---")

#     if not chunks:
#         print("No chunks to create vector store.")
#         return None

#     # Initialize the embedding model from HuggingFace
#     print(f"Loading embedding model: '{EMBEDDING_MODEL}'...")
#     embeddings = HuggingFaceEmbeddings(model_name=EMBEDDING_MODEL)

#     # Create the Chroma vector store
#     # This will process all chunks and store their vector representations.
#     # It will be persisted to disk in the VECTORSTORE_DIR.
#     print(f"Creating vector store in '{VECTORSTORE_DIR}'...")
#     Chroma.from_documents(
#         chunks,
#         embeddings,
#         persist_directory=VECTORSTORE_DIR
#     )
#     print("Vector store created and persisted successfully.")

In [5]:
# # Manually download the dataset first as per the instructions in the function.
# doc_chunks = load_and_chunk_documents()
# if doc_chunks:
#     create_and_store_embeddings(doc_chunks)
# print("\n\t--- Data Ingestion Complete ---")
# print(f"Vector store is ready in the '{VECTORSTORE_DIR}' directory.")

In [6]:
# # --- 1. RAG Agent with Crew.ai --- #
# # Initialize embeddings and vector store once to be used by the tool
# print("Loading embeddings and vector store for RAG tool...")
# embeddings = HuggingFaceEmbeddings(model_name=EMBEDDING_MODEL)
# vectorstore = Chroma(
#     persist_directory=VECTORSTORE_DIR,
#     embedding_function=embeddings
# )
# retiver = vectorstore.as_retriever(search_kwargs={"k": 3}) # Retrieve top 3 results

## Defining Tools

In [7]:
# # Defining retrieval tool
# class RetrieveHRPolicy(BaseTool):
#     name: str = "RetrieveHRPolicy"
#     description: str = "Retrives relevant HR policy information based on the query."

#     def _run(self, query: str) -> str:
#         """Run the retrieval tool to get relevant HR policy information."""
#         # print(f"Retrieving information for query: {query}")
#         results = retiver.invoke(query)

#         if not results:
#             return "No relevant information found in the vector database."
        
#         context = "\n".join([doc.page_content for doc in results])
#         # print("\t--- RAG Tool Finished ---")
#         return f"Retrieved context: \n{context}"

In [8]:
class CodingAgent(BaseTool):
    name: str = "CodingAgent"
    description: str = """A coding agent that can write and execute Python code based on user queries.
    It returns the output of the code if successful, or an error message if an error occurs.
    The query must be a valid Python code block.
    """

    def _run(self, query: str) -> str:
        """Run the coding agent to write and execute Python code based on the query."""
        
        # We need to capture the output, so we redirect stdout to a string buffer.
        old_stdout = sys.stdout
        redirected_output = io.StringIO()
        sys.stdout = redirected_output
        
        try:
            # Execute the user's query as a code block.
            # The 'exec' function can run multi-line statements, including imports and function definitions.
            exec(query)
            
            # Get the output from the redirected buffer.
            output = redirected_output.getvalue()
            
            # Restore the original stdout.
            sys.stdout = old_stdout
            
            # If there's no output, we return a message indicating success.
            if not output.strip():
                return "Code executed successfully with no output."
            
            # Return the captured output.
            return output
            
        except Exception as e:
            # If any error occurs during execution, capture it and return the error message.
            # Restore the original stdout first.
            sys.stdout = old_stdout
            return f"An error occurred while executing the code: {e}"

In [9]:
coading_tool = CodingAgent()

print(coading_tool._run("print('Hello')"))  # Test the tool

Hello



## Initializing LLM

In [10]:
# --- Configuring the LLM --- #
llm = LLM(
    model= MODEL_NAME,
    temperature=0.7
)


In [ ]:
# Instantiate the tools
retrival_tool = CodingAgent()

# define Agent first
agent = Agent(
    role="Coading Agent",
    goal="""Write python code based on user queries and answer executing code.""",
    backstory= """You are a helpful assistant. You can write and execute Python code based on user queries and return the output,
      executing the code.""",
    # verbose=True,
    verbose=False,
    tools=[retrival_tool],
    llm=llm
)

# Define Task
task = Task(
    description=f"""Write clean python code based on the user query.
    User Query: {{query}}""",
    expected_output="return the code like ```CODE:\n``` and the output of the code.",
    agent=agent
)

# Define Crew
crew = Crew(
    agents=[agent],
    tasks=[task],
    # verbose=True
    verbose=False
)

In [15]:
# Checking if Crew is ready
print(crew.kickoff(inputs={"query": "write a python code yo print the sum of first 10 odd numbers."}) )

Here is the Python code to print the sum of the first 10 odd numbers:
```python
sum = 0
for i in range(1, 21, 2):
    sum += i
print(sum)
```
The output of this code is 100.


In [25]:
# Defning Main Function
def main():
    """Main function to kickoff the Crew."""
    print("\n\t--- Starting Crew for RAG Agent ---")
    
    while True:
        user_query = input("Enter your query (or type 'exit' to quit): ")
        print(f"\n🙎‍♂️: {user_query}")
        if user_query.lower() == 'exit':
            print("Exiting the Crew. Goodbye!")
            break
        
        # Kickoff the Crew with the user query
        response = crew.kickoff(inputs={"query": user_query})
        print(f"🤖: {response}")

In [26]:
main()


	--- Starting Crew for RAG Agent ---

🙎‍♂️: How do I request time off and what is the process for approval?
🤖: To request time off, you should follow the designated process outlined in the employee handbook. Typically, this involves submitting a request through an online system or directly to your immediate supervisor or the HR department. The request should include the desired dates and the reason for the time off. Approval is subject to manager discretion and business needs, and you will receive a response confirming or denying your request.

🙎‍♂️: How can I update my personal information, such as address or emergency contacts?
🤖: To update your personal information, such as address or emergency contacts, you should notify the HR department or use the designated employee self-service portal, if available. HR will provide the necessary forms or instructions to make the updates. It is essential to keep personal information current to ensure effective communication and for emergency si